In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv

# from holoviews.operation.datashader import regrid
import skimage

# import skimage.filters
# import skimage.feature
# import scipy.ndimage
from tqdm.auto import tqdm
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose, juxt
from itertools import repeat
from glob import glob

# import cachetools

# import numpy_indexed
import pickle

# import pyarrow as pa
import warnings
import os
from numbers import Integral
from dask.delayed import Delayed

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.projects.sigma_circuits.segmentation as segmentation
import paulssonlab.projects.sigma_circuits.matriarch_stub as matriarch_stub

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
hv.extension("bokeh")

# Config

In [ ]:
dask.config.config["distributed"]["scheduler"]["allowed-failures"] = 6
dask.config.config["distributed"]["worker"]["memory"] = {
    "target": 0.7,
    "spill": 0.8,
    "pause": 0.9,
    "terminate": 0.95,
}

In [ ]:
cluster = LocalCluster(scheduler_options=dict(dashboard_address="0.0.0.0:8787"))
client = Client(cluster)

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="05:00:00",
    memory="6GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/projects/paulssonlab/paulssonlab/src/paulssonlab/projects/sigma_circuits/log",
    cores=1,
    processes=1,
    scheduler_options=dict(dashboard_address="0.0.0.0:8787"),
)
client = Client(cluster)

In [ ]:
cluster

In [ ]:
client.restart()

In [ ]:
# cluster.scale(30)
cluster.adapt(maximum=100)

# Segmentation

In [ ]:
good_positions = (
    np.array([17, 25, 54, 64, 96, 124, 150]) - 1
)  # 1-indexing to 0-indexing

In [ ]:
good_positions2 = (good_positions + np.arange(8)[:, np.newaxis]).T.reshape(-1)

In [ ]:
good_positions2

In [ ]:
nd2_filename = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210324/PWM_flipped.nd2"
# nd2_filename = "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/210204/psc101_bistable.nd2"

In [ ]:
matriarch_stub.get_nd2_reader(nd2_filename).sizes

In [ ]:
%%time
data_graph = segmentation.process(
    nd2_filename, time_slice=slice(None), position_slice=slice(None), delayed=True
)

In [ ]:
%%time
# split up computes so we can gather results from multiple workers
# (otherwise the single worker assembling the dict will run out of memory)
# TODO: use recursive_map(..., levels=?)
# TODO: why doesn't the second-level compute work but the first-level does?
# (segment only gets run after all frames are loaded into memory)
# data_futures = {
#     k: {k2: client.compute(v2) for k2, v2 in v.items()} for k, v in data_graph.items()
# }
data_futures = {k: client.compute(v) for k, v in data_graph.items()}
# data_futures = client.compute(data_graph)

In [ ]:
del data_futures

In [ ]:
# TODO: only gather non-failed futures

In [ ]:
stored.keys()

In [ ]:
103 not in stored

In [ ]:
stored = {}

In [ ]:
for k, v in data_futures.items():
    print(k)
    if k in (104, 115):
        continue
    if k not in stored:
        stored[k] = client.gather(v)

In [ ]:
%%time
dfs = client.gather(data_futures)

In [ ]:
dfs = stored

In [ ]:
failed_pos = {k for k, v in dfs.items() if any(v2 is None for v2 in v.values())}
failed_pos

In [ ]:
df = pd.concat(
    {
        k: pd.concat({k2: v2 for k2, v2 in v.items() if v2 is not None}, names=["t"])
        for k, v in dfs.items()
    },
    names=["pos"],
)

In [ ]:
df

In [ ]:
df.info(memory_usage="full")

In [ ]:
data = {"table": df}

## Save data

In [ ]:
filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329_expt210324flipped.pickle"
# filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329_expt210204nonflipped_psc101.pickle"
with open(filename, "wb") as f:
    pickle.dump(data, f)